<a href="https://colab.research.google.com/github/Amanj5486/Finance_ML/blob/main/Amazon_Tf_basedon_last6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import math
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix, accuracy_score


In [ ]:
df=pd.read_csv('amzn_data.csv',index_col='Date',parse_dates=True)

In [ ]:
data=df.filter(['Adj Close'])
dataset=data.values
training_data_len=math.ceil(len(dataset)*0.8)
dataset

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
scaled_data

In [ ]:
train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])

In [ ]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [11]:
model.fit(x_train,y_train,batch_size=1,epochs=1)

1954/1954 [==============================] - 68s 25ms/step - loss: 0.0012


In [12]:
test_data=scaled_data[training_data_len-60:,:]
x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])

In [13]:
x_test=np.array(x_test)

In [14]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [15]:
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)


In [16]:
rmse=np.sqrt(np.mean(predictions-y_test)**2)
rmse

52.22987535774416

In [17]:
valid=data[training_data_len:]
valid['Predictions']=predictions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
valid

,Adj Close,Predictions
Date,,
2019-06-14,1869.670044,1819.092651
2019-06-17,1886.030029,1831.597534
2019-06-18,1901.369995,1844.174438
2019-06-19,1908.790039,1857.013062
2019-06-20,1918.189941,1869.497314
...,...,...
2021-07-06,3198.010010,3136.130127
2021-08-06,3264.110107,3131.696777
2021-09-06,3281.149902,3133.437500


In [21]:
r2=sklearn.metrics.r2_score(y_test, predictions)
r2

0.9722166406176164